# Introduction

Create a script for downloading INCA data.

* Website only allows for 3 months worth of data to be downloaded at a time.

In [1]:
import requests
import urllib
from pathlib2 import Path
import os.path
import datetime
from dateutil.relativedelta import relativedelta

In [2]:
#ODIR = "/home/skalevag/Documents/NRC P8 water energy and sediment/data/temperature/INCA"
#ODIR = "/home/skalevag/Documents/NRC P8 water energy and sediment/data/precip/INCA"
ODIR = "/home/skalevag/Documents/NRC P8 water energy and sediment/data/radiation/INCA"
ODIR = Path(ODIR)

if not os.path.isdir(ODIR):
    os.makedirs(ODIR)

In [3]:
datetimeformat = "%Y-%m-%d %H:%M"

# variable
#params = "T2M" # air temperature 2m above ground
#params = "RR" # 1-hour precipitation sum
params = "GL" # global radiation 

# gridbox for Ötztal area
gridboxlabel = "oetztal"
lat_min = "46.7"
lat_max = "47.2"
lon_min = "10.5"
lon_max = "11.4"

In [4]:
def makeFilename(start,end):
    filenameHead,filenameEnd = ("incal-hourly","nc")
    s = start.replace("-","").replace(" ","").replace(":","")
    e = end.replace("-","").replace(" ","").replace(":","")
    timeslice = f"{s}-{e}"
    filename = "_".join([filenameHead,params,gridboxlabel,timeslice])+f".{filenameEnd}"
    return filename

def makeURL(start,end,lat_min,lat_max,lon_min,lon_max):
    # make start- and endtime strings
    sd = start.replace(" ","%20")
    ed = end.replace(" ","%20")

    # make gridbox string
    bbox = f"{lat_min},{lon_min},{lat_max},{lon_max}"

    url = f"https://forms.hub.zamg.ac.at/v1/grid/d512d5b5-4e9f-4954-98b9-806acbf754f6/historical?anonymous=true&parameters={params}&start={sd}&end={ed}&bbox={bbox}&output_format=netcdf"

    return url

def makeTimeSlices(year):
    """
    Makes a list of start and end times for a specified year.
    """
    #TODO: make more flexible so that appropriate 3 month timeslices between two dates are made
    #TODO: test if desired start and end are smaller than 3 months (then return only that slice)
    month = 1
    maxMonths = 3

    slices = []
    while month<12:
        dtStart = datetime.datetime(year,month,1,0,0)
        dtEnd = dtStart + relativedelta(months=+maxMonths)

        start = dtStart.strftime(datetimeformat)
        end = dtEnd.strftime(datetimeformat)

        slices.append((start,end))

        month = month + maxMonths
    
    return slices

def downloadData(start,end,lat_min,lat_max,lon_min,lon_max,ODIR):
    # check whether file already exists
    filename = makeFilename(start,end)
    outfile = ODIR.joinpath(filename)

    if not os.path.isfile(outfile):
        url = makeURL(start,end,lat_min,lat_max,lon_min,lon_max)
        r = requests.get(url)
        if str(r) == "<Response [400]>":
            raise requests.HTTPError(f"{r}: The data slice that you requested is too big! ZAMG hub only allows for 3 months data download at a time.")
        ff,html = urllib.request.urlretrieve(url, outfile)
        print(filename, "was downloaded.")
    else:
        print(filename, "has already been downloaded:",outfile)
        
    return outfile

In [5]:
#TODO: make function that can be passed dates, and then makes appropriate slices to be downloaded

# get all in a year
for year in range(2011,2022):

    slices = makeTimeSlices(year)

    for start,end in slices:
        downloadData(start,end,lat_min,lat_max,lon_min,lon_max,ODIR)

incal-hourly_GL_oetztal_201101010000-201104010000.nc has already been downloaded: /home/skalevag/Documents/NRC P8 water energy and sediment/data/radiation/INCA/incal-hourly_GL_oetztal_201101010000-201104010000.nc
incal-hourly_GL_oetztal_201104010000-201107010000.nc has already been downloaded: /home/skalevag/Documents/NRC P8 water energy and sediment/data/radiation/INCA/incal-hourly_GL_oetztal_201104010000-201107010000.nc
incal-hourly_GL_oetztal_201107010000-201110010000.nc has already been downloaded: /home/skalevag/Documents/NRC P8 water energy and sediment/data/radiation/INCA/incal-hourly_GL_oetztal_201107010000-201110010000.nc
incal-hourly_GL_oetztal_201110010000-201201010000.nc has already been downloaded: /home/skalevag/Documents/NRC P8 water energy and sediment/data/radiation/INCA/incal-hourly_GL_oetztal_201110010000-201201010000.nc
incal-hourly_GL_oetztal_201201010000-201204010000.nc has already been downloaded: /home/skalevag/Documents/NRC P8 water energy and sediment/data/rad

In [6]:
#TODO: merge all files from same year
# use cdo?